# Preferential crawling
Preferential crawling is a web crawling strategy where a crawler **prioritizes certain web pages** based on predefined criteria instead of visiting all links indiscriminately. Unlike a standard breadth-first or depth-first crawler, which processes URLs in the order they are discovered, a preferential crawler gives higher priority to pages that match specific characteristics.  

<img src="https://i.ibb.co/sp2ZG5BZ/Posnetek-zaslona-2025-03-03-ob-12-45-11.png" alt="Posnetek-zaslona-2025-03-03-ob-12-45-11" border="0">

> Image from the paper Shrivastava et al.: A Study of Focused Web Crawling Techniques

### **How Preferential Crawling Works**  

1. **Seed URLs** – The crawler starts with an initial set of URLs to visit.  
2. **Fetching & Parsing** – It retrieves web pages and extracts links.  
3. **Prioritization** – Links are **ranked and queued** based on a preference mechanism, such as:  
   - **Keyword Matching** – Pages containing certain keywords in their URL, metadata, or content.  
   - **Domain Preferences** – Prioritizing links from a specific domain or set of domains.  
   - **Page Importance** – Using metrics like PageRank, backlinks, or update frequency.  
   - **Topic Relevance** – Filtering pages using Natural Language Processing (NLP) or machine learning.  
4. **Crawling Execution** – The crawler processes the prioritized URLs first before moving to lower-priority links.  
5. **Loop & Termination** – The process continues until a predefined condition is met (e.g., max pages reached, time limit, or no new prioritized links).  

## A simple example of a preferential webcrawler
This example works by defining a keyword we are interested in and aranging the links in the queue based on how well they match the target word. For now we are only checking whether the link tag contains the keyword and assigning hign priprity to links with the given keyword and low priority to others.

In this exercise we will adapt this crawler to use more sophisticated techniques for ranking the links in its queue.

> Does this code recalculate priorities when a link is rediscovered?

In [ ]:
from urllib.parse import urlsplit
import requests
from bs4 import BeautifulSoup
import heapq

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
class PreferentialWebCrawler:
    def __init__(self, seed_url, keyword, max_pages=10):
        self.seed_url = seed_url
        self.keyword = keyword
        self.max_pages = max_pages
        url_parts = urlsplit(seed_url)
        self.domain = url_parts.scheme + "://" + url_parts.netloc
        self.visited = set()
        self.queue = []  # Priority queue (min-heap)
        heapq.heappush(self.queue, (0, seed_url))  # Start with the seed URL (highest priority)

    def fetch_page(self, url):
        """Fetch page content from a URL."""
        try:
            response = requests.get(url, timeout=5)
            if response.status_code == 200:
                return response.text
        except requests.RequestException:
            pass
        return None

    def extract_links(self, html, base_url):
        """Extract and return all links from the HTML content."""
        soup = BeautifulSoup(html, "html.parser")
        links = []
        for a_tag in soup.find_all("a", href=True):
            href = a_tag["href"]
            if href.startswith("http"):
                links.append((href, a_tag))
            elif href.startswith("//"):
                links.append(("https:" + href, a_tag))
            elif href.startswith("/"):
                links.append((base_url + href, a_tag))
        return links

    def in_domain(self, url):
        return url.startswith(self.domain)

    def priority(self, html, link, link_tag):
        """
        Compute the priority of a link.

        Args:
            html (str): HTML content of the page.
            link (str): Link URL.
            link_tag (bs4.Tag): BeautifulSoup tag representing the link.

        Returns:
            float: Priority score (lower number represents high priority).
        """
        priority = 0 if self.keyword in link else 1  # Assign priority (0 = high, 1 = lower)
        return priority

    def crawl(self):
        """Crawl pages, prioritizing links containing the keyword."""
        pages_crawled = 0

        while self.queue and pages_crawled < self.max_pages:
            priority, url = heapq.heappop(self.queue)  # Get the highest-priority URL
            if url in self.visited:
                continue
            
            if not self.in_domain(url):
                continue

            print(f"Crawling (Priority {priority}): {url}")
            html = self.fetch_page(url)
            if not html:
                print("  - Not found")
                continue

            self.visited.add(url)
            pages_crawled += 1

            url_parts = urlsplit(url)
            base_url = url_parts.scheme + "://" + url_parts.netloc
            links = self.extract_links(html, base_url)
            print("  - Found", len(links), "links")
            for link, link_tag in links:
                if link not in self.visited:
                    priority = self.priority(html, link, link_tag)
                    heapq.heappush(self.queue, (priority, link))


seed = "https://en.wikipedia.org/wiki/Albert_Einstein"  # Replace with an actual URL
keyword = "Nobel"  # Prioritize links containing this keyword
crawler = PreferentialWebCrawler(seed, keyword, max_pages=5)
crawler.crawl()


### **Improving the Priority Function**  

In this example, we are only checking if the URL link includes a keyword to determine its priority. There are many situations in which this approach is not good enough. A URL alone might not fully represent the relevance or quality of the content. For example, important information might be hidden in the page text, metadata, or title, rather than in the URL itself. Additionally, some high-quality pages might not contain the keyword in the URL but still offer highly relevant content.  

A more advanced priority function could consider additional factors such as:  

- **Page Content**: Checking if the keyword appears in the text or headings.  
- **Meta Tags**: Analyzing meta descriptions and titles for keyword matches.  
- **Link Popularity**: Prioritizing pages with more inbound links.  
- **URL Depth**: Giving higher priority to links closer to the seed URL.  
- **Page Freshness**: Favoring recently updated content.  

By combining multiple factors, the crawler can make better decisions about which pages to visit first, improving both efficiency and relevance.

In [ ]:
# An example of unsuccessfull crawl
seed = "https://en.wikipedia.org/wiki/Albert_Einstein"  # Replace with an actual URL
keyword = "president"  # Our goal was to see societies that einstein was the presitent of
crawler = PreferentialWebCrawler(seed, keyword, max_pages=5)
crawler.crawl()

### **Checking Surrounding Text**  

When determining the link priority, we cannot use the content of the target website, as it has not been downloaded yet. Because of that, we will use only the information already available on the current website.

The most common way to compute the priority more accurately is to consider the text around the link on the webpage. The surrounding text often provides important context about the linked page, helping to estimate its relevance.

By analyzing this context, the crawler can assign higher priority to links with surrounding text that matches predefined keywords or topics. This method improves precision, especially when URLs alone do not reveal the nature of the content. Additionally, weighting the importance of anchor text or heading tags (such as `<h1>` or `<h2>`) can further refine the priority calculation.  

> In the next example, we will observe link neighbourhood and check if it contains the target keyword.

In [ ]:
class PreferentialWebCrawler2(PreferentialWebCrawler):
    def priority(self, html, link, link_tag):
        """
        Compute the priority of a link.

        Args:
            html (str): HTML content of the page.
            link (str): Link URL.
            link_tag (bs4.Tag): BeautifulSoup tag representing the link.

        Returns:
            float: Priority score (lower number represents high priority).
        """
        window_size = 50
        # Get the content of the parent tag to the link
        sourounding_text = link_tag.parent.text
        index = sourounding_text.find(link_tag.text)
        start = max(0, index - window_size)
        end = min(len(sourounding_text), index + window_size)
        sourounding_text = sourounding_text[start:end]
        
        priority = 0 if self.keyword.lower() in sourounding_text.lower() else 1  # Assign priority (0 = high, 1 = lower)
        return priority

seed = "https://en.wikipedia.org/wiki/Albert_Einstein"  # Replace with an actual URL
keyword = "Nobel"  # Prioritize links containing this keyword
crawler = PreferentialWebCrawler2(seed, keyword, max_pages=5)
crawler.crawl()

We see that we get completely irrelevant pages (worse than before). This happenes because all links that are somewhat close to the target keyword now have high priority. For example in the following paragraph all links would have high priority even if they do not talk directly about the nobel prize.
> <img src="https://i.ibb.co/TB3rdCSn/Posnetek-zaslona-2025-03-03-ob-10-36-24.png" alt="" border="0">

To fix this, we need to consider how close the keyword is to the link in question. We will update the priority function, to evaluate the priority based on distance between keyword an a link.

In [ ]:
class PreferentialWebCrawler2(PreferentialWebCrawler):
    def priority(self, html, link, link_tag):
        """
        Compute the priority of a link.

        Args:
            html (str): HTML content of the page.
            link (str): Link URL.
            link_tag (bs4.Tag): BeautifulSoup tag representing the link.

        Returns:
            float: Priority score (lower number represents high priority).
        """
        window_size = 50
        # Get the content of the parent tag to the link
        sourounding_text = link_tag.parent.text
        index = sourounding_text.find(link_tag.text)
        start = max(0, index - window_size)
        end = min(len(sourounding_text), index + window_size)
        sourounding_text = sourounding_text[start:end]
        
        # Assign priority based on how close to the link the keyword was found (lower number -> higher priority)
        keyword_distance = abs(sourounding_text.lower().find(self.keyword.lower()) - sourounding_text.find(link_tag.text))
        priority = keyword_distance if self.keyword.lower() in sourounding_text.lower() else window_size*2

        return priority

seed = "https://en.wikipedia.org/wiki/Albert_Einstein"  # Replace with an actual URL
keyword = "Nobel"  # Prioritize links containing this keyword
crawler = PreferentialWebCrawler2(seed, keyword, max_pages=5)
crawler.crawl()

Now we get the nobele prize in physics, which was our goal.

Let's try the example from before, where we were searching for societies where, Einstein was the president.

In [ ]:
seed = "https://en.wikipedia.org/wiki/Albert_Einstein"  # Replace with an actual URL
keyword = "president"  # Prioritize links containing this keyword
crawler = PreferentialWebCrawler2(seed, keyword, max_pages=5)
crawler.crawl()

This time, the priority of the link, we are interested in would be >0:

> <img src="https://i.ibb.co/N2GDkwDm/crawling.png" alt="crawling" border="0">

However; there are other mentions of the president keyword, that get higher priority and burry the link we want:

> <img src="https://i.ibb.co/byZYjjn/Posnetek-zaslona-2025-03-03-ob-10-24-29.png" alt="" border="0">

One way to solve the problem of buried links (links that are relevant but located deep within a website's structure) we can use an N-best first crawler instead of a best-first crawler. The approach involves balancing exploration and exploitation to ensure that potentially valuable but buried links are not overlooked.

A **best-first** crawler prioritizes URLs based on their relevance score, always expanding the most promising link first, which can cause it to focus on highly relevant regions and miss deeply buried links. In contrast, an **N-best first crawler** expands the top N highest-scoring URLs in parallel, balancing exploration and exploitation to increase the likelihood of discovering relevant content across different parts of the website.
Once the N-best links are explored, it uses merge sort to include the newly found links in the priority queue.

In [ ]:
# TODO for practice, update the crawler, to support N-best first strategy and check if it finds the target link

### **Using Bag of Words for Priority Computation**  

Another approach is to use a more descriptive representation of the target page than a simple keyword. Instead of relying on a single word, we can compare multiple words from the link’s surrounding text to a more detailed **target description**. This method allows the crawler to prioritize links whose context is more semantically similar to the desired content.

In this example, we will expand the priority function to search for the phrase: **"Society where Einstein was elected president in 1916"**. The crawler will extract the surrounding text (neighborhood) around each link and represent it as a **Bag of Words (BoW)** vector.

### **What are Bag of Words Vectors?**  

Bag of Words is a simple yet powerful text representation method. It converts a piece of text into a vector by counting how often each word appears. The order of the words is ignored, and only the frequency matters. For example, the sentence:

**"Einstein was a famous scientist"**  

Would be represented by the vector:  

| Word       | Count |
|------------|-------|
| Einstein   | 1     |
| was        | 1     |
| a          | 1     |
| famous     | 1     |
| scientist  | 1     |
| some       | 0     |
| other      | 0     |
| words      | 0     |


In [ ]:
# Compute bag of words vector for a simple example
vectorizer = CountVectorizer()
texts = ["This is one example sentence.", "An example sentence is a sentence that serves as an example"]
word_vectors = vectorizer.fit_transform(texts)
dense_matrix = word_vectors.toarray()

print("\nWord Frequency Table:")
for i, text in enumerate(texts):
    print(f"Text {i+1}: {text}")
    for j, word in enumerate(vectorizer.get_feature_names_out()):
        print(f"  {word}: {dense_matrix[i, j]}")
    print()

### **Computing Similarity**  

To compare the **neighborhood text** with the **target description**, we will:  

1. Convert both texts into Bag of Words vectors.
2. Use **cosine similarity** to measure how similar the two vectors are.
3. Assign higher priority to links with a higher similarity score.  

Cosine similarity ranges from **0 (completely different)** to **1 (identical)**, making it a good metric for text comparison.  
Given two vectors **A** and **B**, the **cosine similarity** is calculated as:


$\text{cosine\_similarity}(A, B) = \frac{A \cdot B}{\|A\| \|B\|}$

Where:
- $ A \cdot B $ is the **dot product** of vectors **A** and **B**.
- $ \|A\| $ is the **magnitude** (or length) of vector **A**.
- $ \|B\| $ is the **magnitude** (or length) of vector **B**.


In the next section, we will implement the priority function using Bag of Words and cosine similarity.

In [ ]:
# Compute cosine similarity between two vectors
similarity = cosine_similarity(word_vectors[0], word_vectors[1])[0][0]
print(similarity)

In [ ]:
class PreferentialWebCrawler3(PreferentialWebCrawler):
    def priority(self, html, link, link_tag):
        """
        Compute the priority of a link.

        Args:
            html (str): HTML content of the page.
            link (str): Link URL.
            link_tag (bs4.Tag): BeautifulSoup tag representing the link.

        Returns:
            float: Priority score (lower number represents high priority).
        """
        window_size = 50
        # Get the content of the parent tag to the link
        sourounding_text = link_tag.parent.text
        index = sourounding_text.find(link_tag.text)
        start = max(0, index - window_size)
        end = min(len(sourounding_text), index + window_size)
        sourounding_text = sourounding_text[start:end]

        # Create Bag of Words representations
        vectorizer = CountVectorizer(stop_words='english')
        texts = [self.keyword, sourounding_text]
        word_vectors = vectorizer.fit_transform(texts)

        # Compute cosine similarity between the two bags of words
        similarity = cosine_similarity(word_vectors[0], word_vectors[1])[0][0]
        
        # compue priority based on vector similarity (more similar texts should result in higher priority (lower return number))
        priority = 1 - similarity
        return priority


seed = "https://en.wikipedia.org/wiki/Albert_Einstein"  # Replace with an actual URL
keyword = "Society where Einstein was elected president in 1916"  # Prioritize links containing this keyword
crawler = PreferentialWebCrawler3(seed, keyword, max_pages=5)
crawler.crawl()

### Advanced techniques for implementing preferential crawlers
The goal of this notebook was to show the basic idea behind preferential crawling systems. In practice there are many ways of implementing the strategy for selecting best websites to crawl.

One general method that can be combined with a variety of techniques for aproximating link relevance is to use a **Bayesian classifier** to guide the crawling process. The classifier is trained on example pages from relevant categories in the taxonomy. It predicts the probability that a newly crawled page belongs to one of the desired categories.  

For each crawled page $p$, the classifier computes the probability $Pr(c|p)$ that the page belongs to category $c$. A **relevance score** is then assigned to the page:  

$R(p) = \sum_{c \in c^*} Pr(c|p)$

Where $c^*$ is the set of categories of interest selected by the user.  

The crawler uses this score in two possible strategies:  

- **Soft Focus Strategy**: The relevance score is used as the priority value for all outgoing links from the page. Higher scores lead to higher priority in the frontier queue.  
- **Hard Focus Strategy**: Only links from pages classified into the desired categories or their subcategories are added to the frontier, while others are discarded.  

Additionally, **distiller algorithms** can identify **topical hubs**—pages with many links to authoritative sources on the target topic. These hubs help the crawler discover high-quality content more efficiently.  

By combining Bayesian classifiers with link analysis, preferential crawlers can significantly improve the quality and relevance of the pages they retrieve.